In [1]:
from nsga2.estimator import NSGAIIRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('../data/lexicase_paper/d_yacht.txt', sep=',')

# DEAP interface requires X and y to be numpy arrays, not pandas dataframes
X = df.drop('label', axis=1).values
y = df['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y)

estimator = NSGAIIRegressor(**{
    'pop_size'        : 4, 
    'max_gen'         : 10,
    'max_depth'       : 8,  # 8
    'max_size'        : 64, # 75
    'objectives'      : ['error', 'size'],
    'cx_prob'         : 1/7,
    'initialization'  : 'uniform',
    'pick_criteria'   : 'error', # error, MCDM
    'validation_size' : 0.33,
    'simplify'        : True,
    'simplification_method' : 'bottom_up',
    'verbosity'       : 1,
    'functions'       :[
        'div', 'add', 'sub', 'mul',
        'maximum', 'minimum',
        'sin', 'cos', 'tan',
        'sqrtabs', 'log1p', 'log', 'exp', 'square', 'abs'
    ]
}).fit(X_train, y_train)

gen	evals	best_size	n_simplifications	n_new_hashes	avg train error	avg train size	avg val error	avg val size	med train error	med train size	med val error	med val size	std train error	std train size	std val error	std val size	min train error	min train size	min val error	min val size	max train error	max train size	max val error	max val size
0  	4    	12       	6                	15          	               	              	             	            	               	              	             	            	               	              	             	            	               	              	             	            	               	              	             	            
1  	4    	10       	5                	17          	               	              	             	            	               	              	             	            	               	              	             	            	               	              	             	            	               	              	    

In [2]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse

model      = str(estimator.best_estimator_).replace("ARG", "x_")
size       = len(estimator.best_estimator_)
complexity = size
depth      = estimator.best_estimator_.height

print(model)
print(size)
print(complexity)
print(depth)

for metric, fn, (data_X, data_y) in [
    ('train_r2',  r2_score, (X_train, y_train)),
    ('test_r2',   r2_score, (X_test,  y_test )),
    ('train_mse', mse,      (X_train, y_train)),
    ('test_mse',  mse,      (X_test,  y_test )),
]:
    score = np.nan
    try:
        score = fn(estimator.predict(data_X), data_y)
        print(f"{metric} : {score}")
    except ValueError:
        print(f"(Failed to calculate {metric}")

add(multiply(log1p(maximum(x_2, x_3)), x_3), add(absolute(log1p(exp(add(sin(x_2), x_4)))), x_5))
16
16
7
train_r2 : -199.12432456847966
test_r2 : -223.92050859510042
train_mse : 233.2110798478112
test_mse : 216.3169917974636


In [3]:
print( len(list(estimator.simplifier.pop_hash.keys())) )

n_keys =  len(list(estimator.simplifier.pop_hash.keys()))

for key in list(estimator.simplifier.pop_hash.keys())[:n_keys]:
    print(key)
    for ind in estimator.simplifier.pop_hash[key]:
        print(" -", ind)

# 11001111011111110110111010010100
# 11010101000100110111001000101100

90
01011100010111011100010001111101
 - ARG0
00001111001111100010010100100001
 - ARG1
 - minimum(ARG4, ARG1)
11100110101110100110011001000011
 - ARG2
10101000000111101110001101011110
 - ARG3
 - absolute(ARG3)
 - absolute(ARG3)
 - maximum(cdiv(ARG2, ARG4), ARG3)
 - maximum(log1p(maximum(ARG2, ARG3)), ARG3)
01100111001110100110111011000001
 - ARG4
 - maximum(ARG4, ARG5)
10111011111110101111111001001111
 - ARG5
 - absolute(ARG5)
 - maximum(ARG0, ARG5)
 - minimum(ARG5, ARG5)
00000000000000000000000000000000
 - 10.368506493506494
 - exp(subtract(log1p(ARG2), exp(11.454081956170242)))
01000100000001010000000110110000
 - cos(ARG5)
 - cos(ARG5)
 - cos(ARG5)
 - cos(ARG5)
 - cos(ARG5)
 - cos(ARG5)
 - cos(ARG5)
 - cos(ARG5)
 - cos(ARG5)
 - cos(ARG5)
 - cos(ARG5)
 - cos(ARG5)
 - cos(ARG5)
 - cos(ARG5)
 - cos(ARG5)
 - cos(ARG5)
 - cos(ARG5)
 - cos(ARG5)
 - cos(ARG5)
10100010001110100110011001000010
 - maximum(ARG2, ARG3)
 - maximum(ARG2, ARG3)
 - maximum(ARG2, ARG3)
 - maximum(ARG2, ARG3)
 - maximum